## Preparation

This section is where we prepare for the project, through a variety of initial steps. The steps in this section are as follows:

- Importing Packages
- Importing Data
- Dropping NA Values
- Subsetting Data

### Importing Packages

In [50]:
#Data management
import pandas as pd
import numpy as np
np.random.seed(0)
from pandas_profiling import ProfileReport

#TextBlob Features
from textblob import TextBlob

#Plotting
import matplotlib.pyplot as plt

#SciKit-Learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#nltk
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')

#Tensorflow / Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

#Test
from collections import Counter

### Importing Data

In [51]:
#Training Data
path = "../input/twitter-entity-sentiment-analysis/twitter_training.csv"
train_df = pd.read_csv(path, names=["Tweet_ID", "Entity", "Sentiment", "Tweet_Content"])

#Test Data (Not to be used until the full model has been trained)
test_path = "../input/twitter-entity-sentiment-analysis/twitter_validation.csv"
test_df = pd.read_csv(test_path, names=["Tweet_ID", "Entity", "Sentiment", "Tweet_Content"])

### Dropping NA Values

Here, we drop any rows with null values, as these miss out on key information.

In [52]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [53]:
train_df.shape

### Subsetting Data

As this dataset is quite large, during the exploration process we begin by subsetting the data during the training process, to speed up any testing steps. We also reset the index of the dataframe, to make adding columns easier later on in the process.

In [54]:
df = train_df.sample(frac=0.1)
df.reset_index(drop=True, inplace=True)

In [55]:
df

## Data Exploration

Here, we explore the data, testing if it is balanced, and checking for patterns in missing rows. This can generally be done in an automated fashion with pandas-profiling. The sections under this header include:

- Basic visualisation
- Automated Data Exploration with pandas-profiling
- Checking for balance in output categories

### Basic Visualisation

We can display basic statistics about the data using pandas, and also view a few entries of the dataset, to see example points with which we'll work.

In [56]:
#See overall information about the data frame
df.info()

In [57]:
#Display a few entries
df.head()

### Automated Data Exploration with Pandas-Profiling

Pandas-profiling is a library used to automatically explore data. This gives us a good overview of the dataset, which we can use to inform our later work.

In [58]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile

### Checking Balance of the Data in Output Categories

We want to check the balance of the output column (Sentiment), such that we don't train a model that always predicts one output. This model might have a high accuracy, but we wouldn't have learned anything about trends in the data, other than the count in the most common sentiment. It might be tempting to think about balancing the test data too, but remember that data in the real world will be unlikely to come nicely balanced, and test data is analagous to real world data


The proportion of sentiments ranges from about 0.15 to 0.3, which is generally good balance, such that we are unlikely to see a scenario in which only one class is predicted. We will however be looking out for if our training accuracy forms a plateau at about 0.3, which could be indicative of this problem.

In [59]:
#Checking balance of target classes
sentiments = df["Sentiment"].value_counts()
plt.bar(sentiments.index, sentiments)

## Basic Data Enrichment

Here, we consider basic features that can enhance the dataset. This will include one-hot encoding of the categorical "Entity" variable. We can also use pre-trained NLP systems here to extract features from the text, such as the one from TextBlob, giving us a "polarity" and "subjectivity" value from any text that we give it. These extra features are added to the dataframe.

We can also create an indexer to convert sentiments from labels to indexes, and back again. This is useful in understanding our predictions later on.

Sections under this header include:

- One-Hot Encoding
- Enrichment with Pre-Trained NLP Models (TextBlob)
- Labels to Indexes (and Back Again)

### One-Hot Encoding

Here, we create a Boolean column for each possible entity, as there are only 32 of these. 32 columns are added, each signifying if the Tweet was related to the given entity. Each Tweet will only be related to one entity in total.

In [60]:
#View all possible entities
print(df["Entity"].unique())
print(len(df["Entity"].unique()))

In [61]:
#One-hot encode using Pandas' get_dummies()
onehot = pd.get_dummies(df["Entity"], prefix="Entity")

#Join these new columns back into the DataFrame
df = df.join(onehot)

#Display a sample of the data with our new columns
df.head()

In [62]:
#Remove the Entity column, as we have transformed this data into a more usable format

df = df.drop("Entity", axis=1)

### Enrichment with Pre-Trained Models (TextBlob)

Here, we use the built-in functionality of TextBlob to add dimensionality to the data, by using it to analyse the text of the "Tweet_Content" column, and storing the outputs in a new column. This pre-trained model allows us to easily analyse strings of text, without any of the more involved NLP methods. Though we also plan to use more methods for NLP, this step can be a good to see if this model picks up on areas that our created one does not.

In [63]:
#Adding dimensions with textblob
def tb_enrich(ls):
    #Enriches a column of text with TextBlob Sentiment Analysis outputs
    tb_polarity = []
    tb_subject = []

    for tweet in ls:
        tb_polarity.append(TextBlob(tweet).sentiment[0])
        tb_subject.append(TextBlob(tweet).sentiment[1])
    

    return tb_polarity, tb_subject
    
df["Polarity"], df["Subjectivity"] = tb_enrich(list(df["Tweet_Content"]))

In [64]:
df.head()

### Labels to Indexes (and Back Again)

Here, as discussed above, we create a dictionary in order to change the text in the "Sentiment" column into an index representing the predicted sentiment.

We then convert the "Sentiment" column in the training data to the labels, which is what we will learn to predict

In [65]:
#Define the indexing for each possible label in a dictionary
class_to_index = {"Neutral":0, "Irrelevant":0, "Negative":2, "Positive": 3}

#Creates a reverse dictionary
index_to_class = {0:"Neutral/Irrelevant", 2:"Negative", 3:"Positive"}

#Creates lambda functions, applying the appropriate dictionary
names_to_ids = lambda n: np.array([class_to_index.get(x) for x in n])
ids_to_names = lambda n: np.array([index_to_class.get(x) for x in n])

In [66]:
#Test each function
print(names_to_ids(["Positive", "Negative", "Irrelevant", "Neutral"]))
print(ids_to_names([0,1,2,3]))

In [67]:
#Convert the "Sentiment" column into indexes
df["Sentiment"] = names_to_ids(df["Sentiment"])

In [68]:
df.head()

## NLP Data Enrichment

In this section, we work with the text in the "Tweet_Content" column, extracting all the insight from this that we can, and converting it into a useable format for the neural network that we will train later. This involves removing words that have little meaning (stop word removal), and grouping words with the same meaning regardless of details such as tense (lemmatisation). Then, we use tokenisation in order to codify the presence of words into a matrix, similar in fashion to one-hot encoding. This is known as a "Bag of Words" method.

Sections under this header include:

- Stop Word Removal and Lemmatisation with NLTK
- Tokenisation
- Adding the Tokenised Strings to the DataFrame

### Stop Word Removal and Lemmatisation with NLTK

Here, we first split each string into its individual words, before checking if these:
1. Contain text
2. Are in the list of stopwords

If there is no text in the word, meaning there are only numbers or punctuation (or other characters), or the word is a stop word (words such as "with", "a", "the"), then the word is removed from the string.

We also lemmatise in this step, in which we convert words into their root form, such that tense and other details can be ignored in our final model (a negative statement in the past tense is still negative).

In [69]:
def remove_stopwords(ls):
    #Lemmatises, then removes stop words
    ls = [lemmatiser.lemmatize(word) for word in ls if word not in (stop_english) and (word.isalpha())]
    
    #Joins the words back into a single string
    ls = " ".join(ls)
    return ls

#Splits each string into a list of words
df["Tweet_Content_Split"] = df["Tweet_Content"].apply(word_tokenize)

#Applies the above function to each entry in the DataFrame
lemmatiser = WordNetLemmatizer()
stop_english = Counter(stopwords.words()) #Here we use a Counter dictionary on the cached
                                          # list of stop words for a huge speed-up
df["Tweet_Content_Split"] = df["Tweet_Content_Split"].apply(remove_stopwords)

In [70]:
df.head()

### Tokenisation

We create tokens for the most common words in the dataset, so we can represent the presence of words in our created corpus (the n most common words) with a list of integers. 

In [71]:
#Define the Tokeniser
tokeniser = Tokenizer(num_words=1000, lower=True)

#Create the corpus by finding the most common 
tokeniser.fit_on_texts(df["Tweet_Content_Split"])

In [72]:
#Tokenise our column of edited Tweet content
tweet_tokens = tokeniser.texts_to_matrix(list(df["Tweet_Content_Split"]))

In [73]:
tweet_tokens.shape

### Adding the Tokenised Strings to the DataFrame

Currently, the tokens are contained in a matrix titled "tweet_tokens". We then want to combine these back into the dataframe containing all of the current data. This is completed below, and then we test to make sure that this has occurred correctly by looking at the number of columns compared to that of the original matrices.

In [74]:
print(df.shape)
print(tweet_tokens.shape)

In [75]:
#Combining the dataframe with the tokens using pd.concat
full_df = pd.concat([df, pd.DataFrame(tweet_tokens)], sort=False, axis=1)
full_df.shape

## Final Data Preparation

The data is now almost ready for a model to be trained on it, but a few final preparations will need to occur. For example, we need to drop the columns that we don't plan to use, such as the "Tweet_Content" column, which has had its useful information extracted already.

We also split the data into a training and test set, such that we can evaluate our model's performance without touching the held-out data. We do this because if we continually test against this held-out data, it loses its usefulness as unseen "real-world" data.

Sections under this header include:
- Dropping Unused Data
- Test-Train Split

In [76]:
#Remove dependent variable
y = full_df["Sentiment"]

#Drop all non-useful columns
full_df = full_df.drop(["Sentiment", "Tweet_ID", "Tweet_Content", "Tweet_Content_Split"], axis=1)

### Dropping Unused Data

We drop non-useful columns from the DataFrame here. These either have no use (Tweet ID), or have already had the useful information extracted (Tweet Content). We also remove the "y" or dependent variable here, so we don't accidentally train on it.

### Test-Train Split

Here, we use SciKit-Learn's inbuilt function to split our data into a test set and a train set, with the appropriate labels. We use a constant random state to make this replicable.

In [77]:
X_train, X_test, y_train, y_test = train_test_split(full_df, y, test_size=0.2, random_state=1)

## Model Construction and Training

Finally, it is time to construct our model. In this case, we use a neural network constructed with Keras. We then train it with our data in the training dataset, and validate using the test datasets.

Sections under this header include:
- Model Construction
- Training

### Model Construction

Here, we define the neural network that we will train to predict the output. This model is constructed with the following layers:
1. Dense
2. Dropout

The Dense layers are fully-connected layers. This means that inbetween each layer, we can transfer data from any neuron to any one in the next layer (or indeed all others), scaled by the weight associated with that transfer. These weights are trained.

The Dropout layers prevent our overall weights from getting too large, as can happen with larger neural networks. This helps to stop certain areas of the network from overloading the network as a whole.


In [78]:
#Test model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, input_dim=1034, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(4, activation='sigmoid')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

### Training

Next, we fit this model with our data, using backpropagation, for 30 epochs. We can view the increase in accuracy of the model through the different epochs, on both the training and test dataset.

In [79]:
h = model.fit(
     X_train, y_train,
     validation_data=(X_test, y_test),
     epochs=30,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)]
)

## Model Evaluation

Now that we've trained the model, we can view it's accuracy with a confusion matrix. This allows us to see the predictions for Tweets with various true values. From this, we might see that we are better at predicting certain classes than others, such as in this model, where we can predict Negative and Positive sentiment significantly better than Irrelevant or Neutral.

In [80]:
#Generate predictions
y_pred = np.argmax(model.predict(X_test), axis=1)

#Assign labels to predictions and test data
y_pred_labels = ids_to_names(y_pred)
y_test_labels = ids_to_names(y_test)

In [81]:
y_unique = list(set(y_test_labels))
cm = confusion_matrix(y_test_labels, y_pred_labels, labels = y_unique, normalize='true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=y_unique)
disp.plot()

## Training with Full Dataset

Now that we are happy with our model, we can train using the full dataset, and predict the held-out test data. This involves performing all of our transformation steps on both this training dataset and the held-out test data. Luckily, we can reuse the code from above to achieve this, so little further explanation is required.

In [82]:
#Use the full dataset!
df = train_df

# the test dataframe was loaded earlier and is named test_df

### Basic Data Enrichment

In [83]:
#One-hot encode using Pandas' get_dummies()

##Train
onehot = pd.get_dummies(df["Entity"], prefix="Entity")

#Join these new columns back into the DataFrame
df = df.join(onehot)


##Test
onehot = pd.get_dummies(test_df["Entity"], prefix="Entity")

test_df = test_df.join(onehot)

In [84]:
#Enrich using TextBlob's built in sentiment analysis

##Train
df["Polarity"], df["Subjectivity"] = tb_enrich(list(df["Tweet_Content"]))


##Test
test_df["Polarity"], test_df["Subjectivity"] = tb_enrich(list(test_df["Tweet_Content"]))

In [85]:
#Convert the "Sentiment" column into indexes

##Train
df["Sentiment"] = names_to_ids(df["Sentiment"])
y = df["Sentiment"]

##Test
test_df["Sentiment"] = names_to_ids(test_df["Sentiment"])
y_test = test_df["Sentiment"]

### NLP Data Enrichment

In [86]:
#Removing stopwords and lemmatising

##Train
#Splits each string into a list of words
df["Tweet_Content_Split"] = df["Tweet_Content"].apply(word_tokenize)

#Applies the above function to each entry in the DataFrame
lemmatiser = WordNetLemmatizer()
stop_english = Counter(stopwords.words()) #Here we use a Counter dictionary on the cached
                                          # list of stop words for a huge speed-up
df["Tweet_Content_Split"] = df["Tweet_Content_Split"].apply(remove_stopwords)

##Test
test_df["Tweet_Content_Split"] = test_df["Tweet_Content"].apply(word_tokenize)

test_df["Tweet_Content_Split"] = test_df["Tweet_Content_Split"].apply(remove_stopwords)

In [87]:
#Tokenisation

#Define the Tokeniser
tokeniser = Tokenizer(num_words=1000, lower=True)

#Create the corpus by finding the most common 
tokeniser.fit_on_texts(df["Tweet_Content_Split"])

##Train
#Tokenise our column of edited Tweet content
tweet_tokens = tokeniser.texts_to_matrix(list(df["Tweet_Content_Split"]))

##Test
#Tokenise our column of edited Tweet content
tweet_tokens_test = tokeniser.texts_to_matrix(list(test_df["Tweet_Content_Split"]))

In [88]:
#Combining the dataframe with the tokens using pd.concat

#Reset axes to avoid overlapping
df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

##Train
full_df = pd.concat([df, pd.DataFrame(tweet_tokens)], sort=False, axis=1)

##Test
full_test_df = pd.concat([test_df, pd.DataFrame(tweet_tokens_test)], sort=False, axis=1)

In [89]:
#Final prep

##Train
#Drop all non-useful columns
full_df = full_df.drop(["Sentiment", "Tweet_ID", "Tweet_Content", "Tweet_Content_Split", "Entity"], axis=1)


##Test
full_test_df = full_test_df.drop(["Sentiment", "Tweet_ID", "Tweet_Content", "Tweet_Content_Split", "Entity"], axis=1)

### Model Definition and Training

This time, we train with all of the available training data

In [90]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, input_dim=1034, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(4, activation='sigmoid')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [91]:
h = model.fit(
     full_df, y,
     epochs=30,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)]
)

## Final Model Evaluation

In [92]:
#Generate predictions
y_pred = np.argmax(model.predict(full_test_df), axis=1)

#Assign labels to predictions and test data
y_pred_labels = ids_to_names(y_pred)
y_test_labels = ids_to_names(y_test)

In [93]:
y_unique = list(set(y_test_labels))
cm = confusion_matrix(y_test_labels, y_pred_labels, labels = y_unique, normalize='true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=y_unique)
disp.plot()

In [94]:
#To see the final accuracy
accuracy_score(y_test, y_pred)